In [10]:
import math
from dateutil import relativedelta
from datetime import datetime, timedelta

import rivapy
from rivapy.marketdata import DiscountCurve, DatedCurve, SurvivalCurve
from rivapy import enums

In [11]:
#Specs
refdate = datetime(2020,1,1)
premium_payment = 0.0012
notional_amount=1000000

#yield curve
days_to_maturity = [1, 180, 360, 720, 3*360, 4*360, 5*360, 10*360]
rates = [-0.0065, 0.0003, 0.0059, 0.0086, 0.0101, 0.012, 0.016, 0.02]
dates = [refdate + timedelta(days=d) for d in days_to_maturity]
dsc_fac = [math.exp(-rates[i]*days_to_maturity[i]/360) for i in range(len(days_to_maturity))]
dc = DiscountCurve('CDS_interest_rate', refdate, 
                                    dates, 
                                    dsc_fac,
                                    enums.InterpolationType.LINEAR)
hazard_rates = [0, 0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.005]

#SurvivalCurve
sc = SurvivalCurve('Survival', refdate, dates, hazard_rates)

#RecoverCurve
recoveries = [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]
recovery = DatedCurve('Recovery', refdate, dates, recoveries,
                                        enums.DayCounterType.Act365Fixed.name,
                                        enums.InterpolationType.LINEAR.name)

#Coupon Payment Dates
payment_dates = [refdate + relativedelta.relativedelta(years=i) for i in range(10)]

#RiVaPy Spec
spec = rivapy.instruments.CDSSpecification(premium = premium_payment, protection_start=refdate, premium_pay_dates = payment_dates, notional = notional_amount)

#Pricind Data
cds_pricing_data = rivapy.pricing.CDSPricingData(spec=spec, val_date=refdate, discount_curve=dc, survival_curve=sc, recovery_curve=recovery)

#Price
pr = rivapy.pricing.price(cds_pricing_data)




In [12]:
print("Expected PV:", pr._price)
print("Expected PV protection leg:",pr.pv_protection)
print("Expected PV premium leg:",pr.premium_leg)
print("Expected PV accrued interest:", pr.accrued)

Expected PV: 1807.3975359561114
Expected PV protection leg: 11647.702645148813
Expected PV premium leg: 9822.995285850662
Expected PV accrued interest: 17.309823342039067


In [13]:
pr.par_spread

AttributeError: 'PricingResults' object has no attribute 'par_spread'